In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import os
import sys
import json
import copy
import pickle
from hashlib import sha256
import random

In [3]:
import pandas as pd
import pandas as pd
import numpy as np
import cv2
from tqdm.notebook import tqdm

In [4]:
sys.path.append('../lared')
from helpers import file_functions
from dataset.example import VideoExample, AudioExample, FullExample
from constants.laughter_constants import data_path, cloud_data_path
from data_loading.utils import get_video_caps, AudioFiles

In [5]:
VideoExample.init_caps(get_video_caps('/mnt/hdd/data/lared/concat'))

TypeError: get_video_caps() takes 0 positional arguments but 1 was given

In [5]:
laughter_ann_path = os.path.join(cloud_data_path, 'laughter.txt')
camera_ann_path = os.path.join(cloud_data_path, 'camera.txt')
out_path = os.path.join(cloud_data_path, 'examples')
images_path = os.path.join(out_path, 'images')


laughter_ann = pd.read_csv(laughter_ann_path, sep='\t',names=['name','participant','ini','fin','dur','type'],header=None)
camera_ann = pd.read_csv(camera_ann_path, sep='\t',names=['name','participant','ini','fin','dur','camera'],header=None)

In [6]:
camera_ann.head()

,name,participant,ini,fin,dur,camera
0,camera30,32.0,0.000,10.000,10.000,0
1,camera3,3.0,866.280,882.440,16.160,1
2,camera3,3.0,883.150,884.490,1.340,12
3,camera3,3.0,884.600,888.860,4.260,123
4,camera3,3.0,4300.139,4300.956,0.817,4


In [7]:
df_camera_by_participant = camera_ann.groupby('participant')
df_laughter_by_participant = laughter_ann.groupby('participant')

In [8]:
dfs_camera = []
for i in df_camera_by_participant.groups:
    dfs_camera.append(df_camera_by_participant.get_group(i))
dfs_laughter = []
for i in df_laughter_by_participant.groups:
    dfs_laughter.append(df_laughter_by_participant.get_group(i))
participant_ids = df_camera_by_participant.groups.keys()
participant_ids = [int(id) for id in participant_ids]

In [9]:
def get_camera(camerastr):
    if int(camerastr) == 0:
        return []
    
    return [int(c) for c in str(camerastr)]

In [10]:
# find the camera where a person is seen
def find_cameras(cam, t):
    previous = cam[cam['fin'] < t]
    if len(previous) == 0:
        return []
    
    row = previous.iloc[-1]
    cameras = row.tail()['camera']
    return get_camera(cameras)

# Laughter examples

In [12]:
# Create examples and first images
examples = dict()
for pid, df_laughter, df_camera in zip(participant_ids, dfs_laughter, dfs_camera):
    for index,row in df_laughter.iterrows():
        cameras = find_cameras(df_camera, row['ini'])
        ini_time = (row['ini'])
        fin_time = (row['fin'])

        for cam in cameras:

            hashstr = '_'.join([str(pid), str(cam), str(ini_time), str(fin_time)])
            hashid = sha256(hashstr.encode()).hexdigest()
            
            examples[hashid] = FullExample({
                'id': hashid,
                'pid': pid,
                'cam': cam,
                'ini_time': ini_time,
                'fin_time': fin_time
            })

            examples[hashid].write_full_first_image(images_path)

AttributeError: 'FullExample' object has no attribute 'write_full_first_image'

In [11]:
def create_random_segments(df_camera, pid, segment_len=10):
    examples = list()
    it = df_camera.iterrows()
    prev = next(it)[1]
    for index, row in it:
        ini = prev.loc['fin'] + 10
        fin = row.loc['ini'] - 10
        cameras = get_camera(prev.loc['camera'])

        for cam in cameras:
            t = ini
            while t < fin:
                if t < 14*60:
                    t += segment_len
                    continue

                hashstr = '_'.join([str(pid), str(cam), str(t), str(t+segment_len)])
                hashid = sha256(hashstr.encode()).hexdigest()
                examples.append(FullExample({
                    'id': hashid,
                    'pid': pid,
                    'cam': cam,
                    'ini_time': t,
                    'fin_time': t + segment_len
                }))
                t += segment_len
        prev = row
    return examples

In [12]:
random_examples = [create_random_segments(df, pid) for pid, df in zip(participant_ids, dfs_camera)]

In [13]:
random_examples = [ex for e in random_examples for ex in e]

In [14]:
# sample examples and put them in order of pid (for easy labeling)
random.seed(22)
chosen_examples = random.sample(random_examples, 2000)
chosen_examples = sorted(chosen_examples, key=lambda ex: ex.pid)

In [15]:
len(chosen_examples)

2000

# Write images

In [24]:
for ex in tqdm(chosen_examples[:2000]):
    ex.write_full_first_image(images_path)

KeyboardInterrupt: 

# Read CVAT annotations

In [16]:
examples_dict = {ex.id: ex for ex in random_examples}

In [17]:
cvat_bb_path = os.path.join(cloud_data_path, 'examples/pilot/cvat.json')

In [18]:
# Read the BBs from cvat annotations
# Read camera and bounding box from it
with open(cvat_bb_path) as json_file:
    bbs = json.load(json_file)

In [19]:
# create dict of img_id -> img
imgs_dict = dict()
for img in bbs['images']:
    imgs_dict[img['id']] = img

In [20]:
# iterate over annotations and write media
lared_examples = list()
for annot in bbs['annotations']:
    annot_img = imgs_dict[annot['image_id']]
    example_id = annot_img['file_name'].split('_')[1]
    example = examples_dict[example_id]
    lared_examples.append(example)

    bbox = annot['bbox']
    example.rect = [int(e) for e in bbox]

# AMI examples

In [8]:
import random
import xml.etree.ElementTree as ET
from ami_example import AmiExample
from tqdm.notebook import trange, tqdm

In [2]:
ami_annotations = '/Users/josedvq/data/AMI/ami_public_manual_1.6.2/words'
AmiExample.corpus_path = '/Users/josedvq/data/AMI/amicorpus'

In [3]:
meetings_to_use = ['IS1002c']

In [4]:
# load annotations
meeting_annotations = dict()
part_letters = ['', 'A', 'B', 'C', 'D']
for meeting in meetings_to_use:
    meeting_annotations[meeting] = dict()
    for part_num in [1, 2, 3, 4]:
        root = ET.parse(os.path.join(ami_annotations, f'{meeting}.{part_letters[part_num]}.words.xml')).getroot()
        meeting_annotations[meeting][part_num] = {
            'ini': float(root[0].attrib['starttime']),
            'end': float(root[-1].attrib['endtime']),
            'annot': [{
                'type': t.tag,
                'ini': float(t.attrib['starttime']), 
                'end': float(t.attrib['endtime']),
                'content': t.text} 
                for t in root]
        }

In [5]:
def pick_random_segment(annot, length=10):
    ini = int(random.random() * (annot['end'] - annot['ini'] - length))
    return ini, ini + length

In [9]:
# example generation
N = 3
for i in trange(0,N):
    # pick a random meeting
    meeting = random.choice(meetings_to_use)
    random.sample([1,2,3,4], k=1)
    participant = random.choice([1,2,3,4])

    # pick a random meeting segment
    ini, end = pick_random_segment(meeting_annotations[meeting][participant])
    ex = AmiExample(meeting, participant, ini, end)
    ex.write_video('test')
    ex.write_audio('test')
    ex.write_audiovisual('test', 'test', 'test')

# Create covfee files

In [21]:
task = {
    "type": "QuestionnaireTask",
    "form": {
        "fields": [
            {
                "prompt": "Arousal: ",
                "input": {
                  "type": "Slider",
                  "defaultValue": 50
                }
            }
        ]
    },
    "media": {
        "type": "video",
        "autoplay": False,
        "controls": True
    }
}

In [22]:
video_tasks = []
audio_tasks = []
av_tasks = []
for example in lared_examples:
    t = copy.deepcopy(task)
    t['media']['url'] = 'pilot/video/' + example.get_file_id() + '.mp4'
    t['instructions'] = 'Find the person indicated with a red rectangle. Play the video and pay attention to this person. Give the required ratings on the right.'
    video_tasks.append(t)

    t = copy.deepcopy(task)
    t['media'] = {
        'type': 'audio',
        'url': 'pilot/audio/' + example.get_file_id() + '.wav'
    }
    t['instructions'] = 'Play the audio, paying attention to the main speaker. Then give the required ratings on the right.'
    audio_tasks.append(t)

    t = copy.deepcopy(task)
    t['media']['url'] = 'pilot/av/' + example.get_file_id() + '.mp4'
    t['instructions'] = 'Find the person indicated with a red rectangle. Play the video and pay attention to this person. Give the required ratings on the right.'
    av_tasks.append(t)

In [23]:
project = {
    "name": "Pilot",
    "id": "pilot",
    "email": "example@example.com",
    "hits": [
        {
            "id": "pilot",
            "name": "Pilot",
            "type": "timeline",
            "tasks": [
                {
                    "type": "VideoInstructionsTask"
                },
                *video_tasks,
                {
                    "type": "AudioInstructionsTask"
                },
                *audio_tasks,
                {
                    "type": "AVInstructionsTask"
                },
                *av_tasks
            ]
                
        }
    ]
}

In [24]:
json.dump(project, open('pilot.covfee.json', 'w'), indent=2)